In [1]:
options(warn = -1)
options(scipen = 100)
library(nnet)
library(party)
library(randomForest)
library(e1071)
library(caret)
library(dplyr)
library(Metrics)

Loading required package: grid
Loading required package: mvtnorm
Loading required package: modeltools
Loading required package: stats4
Loading required package: strucchange
Loading required package: zoo

Attaching package: 'zoo'

The following objects are masked from 'package:base':

    as.Date, as.Date.numeric

Loading required package: sandwich
randomForest 4.6-14
Type rfNews() to see new features/changes/bug fixes.
Loading required package: lattice
Loading required package: ggplot2
Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang

Attaching package: 'ggplot2'

The following object is masked from 'package:randomForest':

    margin


Attaching package: 'dplyr'

The following object is masked from 'package:randomForest':

    combine

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, 

In [2]:
# 인공 신경망
f.nnet <- function(data.tr, x, y) {
    model.nnet <- nnet(cluster ~ ., data=data.tr, size = 10) 
    pred <- predict(model.nnet, x, type="class")
    return (mean(pred == y))
}

# 의사 결정 나무
f.ctree <- function(data.tr, x, y) {
    model.ctree <- ctree(cluster ~ ., data=data.tr)
    pred <- predict(model.ctree, x)
    return (mean(pred == y))
}

# RandomForest
f.randomForest <- function(data.tr, x, y) {
    model.rf <- randomForest(cluster ~ ., data=data.tr, ntree=100)
    pred <- predict(model.rf, x)
    return (mean(pred == y))
}


# SVM
f.svm <- function(data.tr, x, y) {
    model.svm <- svm(cluster ~., data=data.tr)
    pred <- predict(model.svm, x)
    return (mean(pred == y))
}

# 다항 로지스틱 회귀분석
f.multinom <- function(data.tr, x, y) {
    model.multinom <- multinom(cluster ~ ., data=data.tr)
    pred <- predict(model.multinom, x)
    return (mean(pred == y))
}


In [3]:
data <- read.csv("./진짜진짜최종데이터셋.csv", fileEncoding = "utf-8")

In [4]:
data$cluster <- as.factor(data$cluster)

In [5]:
result <- rep(0,5*(ncol(data)-1))
temp <- rep(0,5*(ncol(data)-1))
n <- 11
samp <- createDataPartition(data$cluster, p = 0.7, list = F)

for (j in 2:ncol(data)) {
    for (i in 1:n) {
        # 학습용 데이터 셋
        data.tr <- as.data.frame(cbind(i=data[samp, colnames(data)[j]], cluster = data[samp,"cluster"]))

        # 검증용 데이터셋
        data.test <- as.data.frame(cbind(i=data[-samp, colnames(data)[j]], cluster = data[samp,"cluster"]))
        
        data.tr$cluster <- as.factor(data.tr$cluster)
        data.test$cluster <- as.factor(data.test$cluster)

        # 문제 및 정답 만들기
        x <- subset(data.test, select = -cluster)
        y <- data.test$cluster
        result[5*(j-2) + 1] <- result[5*(j-2) + 1] + f.nnet(data.tr, x, y)
        result[5*(j-2) + 2] <- result[5*(j-2) + 2] + f.ctree(data.tr, x, y)
        result[5*(j-2) + 3] <- result[5*(j-2) + 3] + f.randomForest(data.tr, x, y)
        result[5*(j-2) + 4] <- result[5*(j-2) + 4] + f.svm(data.tr, x, y)
        result[5*(j-2) + 5] <- result[5*(j-2) + 5] + f.multinom(data.tr, x, y)

        temp[5*(j-2) + 1] <- temp[5*(j-2) + 1] + f.nnet(data.tr, subset(data.tr, select = -cluster), data.tr$cluster)
        temp[5*(j-2) + 2] <- temp[5*(j-2) + 2] + f.ctree(data.tr, subset(data.tr, select = -cluster), data.tr$cluster)
        temp[5*(j-2) + 3] <- temp[5*(j-2) + 3] + f.randomForest(data.tr, subset(data.tr, select = -cluster), data.tr$cluster)
        temp[5*(j-2) + 4] <- temp[5*(j-2) + 4] + f.svm(data.tr, subset(data.tr, select = -cluster), data.tr$cluster)
        temp[5*(j-2) + 5] <- temp[5*(j-2) + 5] + f.multinom(data.tr, subset(data.tr, select = -cluster), data.tr$cluster)
    }
}
result <- result / n
temp <- temp / n

# weights:  53
initial  value 105.593002 
iter  10 value 81.229001
iter  20 value 78.013073
iter  30 value 77.560938
iter  40 value 77.552332
iter  50 value 77.552152
final  value 77.552140 
converged
# weights:  9 (4 variable)
initial  value 84.593146 
final  value 81.708812 
converged
# weights:  53
initial  value 90.060536 
iter  10 value 81.287718
iter  20 value 78.182484
iter  30 value 77.567200
iter  40 value 77.552331
iter  50 value 77.552146
final  value 77.552139 
converged
# weights:  9 (4 variable)
initial  value 84.593146 
final  value 81.708812 
converged
# weights:  53
initial  value 91.719914 
iter  10 value 81.409779
iter  20 value 79.094212
iter  30 value 77.555927
iter  40 value 77.552202
final  value 77.552138 
converged
# weights:  9 (4 variable)
initial  value 84.593146 
final  value 81.708812 
converged
# weights:  53
initial  value 96.313409 
iter  10 value 81.082823
iter  20 value 78.585273
iter  30 value 77.554466
iter  40 value 77.552174
final  value 77.552139

initial  value 84.593146 
final  value 78.147736 
converged
# weights:  53
initial  value 146.703352 
iter  10 value 82.128910
iter  20 value 78.553728
iter  30 value 78.264771
iter  40 value 78.110596
iter  50 value 77.913448
iter  60 value 77.877990
iter  70 value 77.852350
iter  80 value 77.851356
final  value 77.851307 
converged
# weights:  9 (4 variable)
initial  value 84.593146 
final  value 78.147736 
converged
# weights:  53
initial  value 102.653798 
iter  10 value 80.084603
iter  20 value 78.131692
iter  30 value 77.937429
iter  40 value 77.905100
iter  50 value 77.900085
iter  60 value 77.899314
iter  70 value 77.893629
iter  80 value 77.874464
iter  90 value 77.858403
iter 100 value 77.851680
final  value 77.851680 
stopped after 100 iterations
# weights:  9 (4 variable)
initial  value 84.593146 
final  value 78.147736 
converged
# weights:  53
initial  value 101.562932 
iter  10 value 81.176916
iter  20 value 78.564343
iter  30 value 78.198317
iter  40 value 77.902538
ite

initial  value 84.593146 
final  value 80.354607 
converged
# weights:  53
initial  value 93.796993 
iter  10 value 78.456913
iter  20 value 76.164287
iter  30 value 74.106727
iter  40 value 72.917221
iter  50 value 70.908139
iter  60 value 70.417187
iter  70 value 69.688146
iter  80 value 69.500194
iter  90 value 69.449082
iter 100 value 69.426705
final  value 69.426705 
stopped after 100 iterations
# weights:  9 (4 variable)
initial  value 84.593146 
final  value 80.354607 
converged
# weights:  53
initial  value 104.516696 
iter  10 value 78.253006
iter  20 value 76.075197
iter  30 value 72.397856
iter  40 value 71.787229
iter  50 value 71.764963
iter  60 value 71.673337
iter  70 value 71.632131
iter  80 value 71.565907
iter  90 value 71.486023
iter 100 value 70.884759
final  value 70.884759 
stopped after 100 iterations
# weights:  9 (4 variable)
initial  value 84.593146 
final  value 80.354607 
converged
# weights:  53
initial  value 160.044425 
iter  10 value 78.381373
iter  20 v

initial  value 84.593146 
iter  10 value 63.975461
final  value 63.975288 
converged
# weights:  53
initial  value 126.040677 
iter  10 value 78.570322
iter  20 value 64.273902
iter  30 value 61.194288
iter  40 value 60.231998
iter  50 value 59.573730
iter  60 value 54.167939
iter  70 value 53.301009
iter  80 value 53.270365
iter  90 value 53.265767
iter 100 value 53.264002
final  value 53.264002 
stopped after 100 iterations
# weights:  9 (4 variable)
initial  value 84.593146 
iter  10 value 63.975461
final  value 63.975288 
converged
# weights:  53
initial  value 85.164405 
iter  10 value 70.670083
iter  20 value 64.848790
iter  30 value 63.039517
iter  40 value 61.841418
iter  50 value 60.958172
iter  60 value 57.704832
iter  70 value 52.469197
iter  80 value 46.160956
iter  90 value 42.604248
iter 100 value 42.199683
final  value 42.199683 
stopped after 100 iterations
# weights:  9 (4 variable)
initial  value 84.593146 
iter  10 value 63.975461
final  value 63.975288 
converged
# 

initial  value 84.593146 
iter  10 value 63.975461
final  value 63.975288 
converged
# weights:  53
initial  value 94.042084 
iter  10 value 73.468491
iter  20 value 64.183967
iter  30 value 60.750481
iter  40 value 49.601955
iter  50 value 44.552593
iter  60 value 44.120196
iter  70 value 43.457925
iter  80 value 43.137238
iter  90 value 42.538228
iter 100 value 42.498701
final  value 42.498701 
stopped after 100 iterations
# weights:  9 (4 variable)
initial  value 84.593146 
iter  10 value 63.975461
final  value 63.975288 
converged
# weights:  53
initial  value 98.646808 
iter  10 value 68.894993
iter  20 value 63.853906
iter  30 value 62.217882
iter  40 value 61.249327
iter  50 value 60.401691
iter  60 value 59.824284
iter  70 value 59.111374
iter  80 value 58.277924
iter  90 value 57.648332
iter 100 value 56.780121
final  value 56.780121 
stopped after 100 iterations
# weights:  9 (4 variable)
initial  value 84.593146 
iter  10 value 73.398136
final  value 73.398129 
converged
# w

initial  value 84.593146 
iter  10 value 73.398136
final  value 73.398129 
converged
# weights:  53
initial  value 86.104197 
iter  10 value 68.081069
iter  20 value 63.309708
iter  30 value 61.189857
iter  40 value 60.574558
iter  50 value 60.118753
iter  60 value 59.894783
iter  70 value 59.430497
iter  80 value 59.220084
iter  90 value 58.867016
iter 100 value 58.800120
final  value 58.800120 
stopped after 100 iterations
# weights:  9 (4 variable)
initial  value 84.593146 
iter  10 value 73.398136
final  value 73.398129 
converged
# weights:  53
initial  value 89.970774 
iter  10 value 68.767437
iter  20 value 65.417538
iter  30 value 65.027817
iter  40 value 63.194356
iter  50 value 61.834704
iter  60 value 60.855871
iter  70 value 60.644183
iter  80 value 60.579199
iter  90 value 60.483081
iter 100 value 60.423553
final  value 60.423553 
stopped after 100 iterations
# weights:  9 (4 variable)
initial  value 84.593146 
iter  10 value 73.398136
final  value 73.398129 
converged
# w

initial  value 84.593146 
iter  10 value 46.284442
iter  20 value 46.202872
iter  30 value 46.196216
final  value 46.196029 
converged
# weights:  53
initial  value 85.178819 
iter  10 value 67.462415
iter  20 value 46.049737
iter  30 value 45.690347
iter  40 value 45.271994
iter  50 value 43.712585
iter  60 value 42.033720
iter  70 value 41.125053
iter  80 value 40.088125
iter  90 value 39.793391
iter 100 value 39.480658
final  value 39.480658 
stopped after 100 iterations
# weights:  9 (4 variable)
initial  value 84.593146 
iter  10 value 46.284442
iter  20 value 46.202872
iter  30 value 46.196216
final  value 46.196029 
converged
# weights:  53
initial  value 95.520491 
iter  10 value 53.490814
iter  20 value 46.056670
iter  30 value 45.953759
iter  40 value 45.857544
iter  50 value 44.967709
iter  60 value 41.734419
iter  70 value 40.282405
iter  80 value 38.808285
iter  90 value 36.456703
iter 100 value 36.184155
final  value 36.184155 
stopped after 100 iterations
# weights:  9 (

initial  value 84.593146 
iter  10 value 46.284442
iter  20 value 46.202872
iter  30 value 46.196216
final  value 46.196029 
converged
# weights:  53
initial  value 124.632639 
iter  10 value 54.069961
iter  20 value 45.926608
iter  30 value 45.789019
iter  40 value 45.453400
iter  50 value 43.531686
iter  60 value 42.948779
iter  70 value 41.657642
iter  80 value 40.060920
iter  90 value 38.478225
iter 100 value 37.477886
final  value 37.477886 
stopped after 100 iterations
# weights:  9 (4 variable)
initial  value 84.593146 
iter  10 value 46.284442
iter  20 value 46.202872
iter  30 value 46.196216
final  value 46.196029 
converged


In [6]:
res_all <- rbind(t(as.data.frame(temp)),
t(as.data.frame(result)))
colnames(res_all) <- rep(c("인공신경망", "의사결정나무", "randomForest", "SVM", "다항로지스틱회귀분석"), ncol(data)-1)
temp_all <- c()
for (i in 1:(ncol(res_all)/5)) {
    temp_all <- rbind(temp_all, res_all[,(5*(i-1) + 1):(5*i)])
}
rownames(temp_all) <- paste(rep(colnames(data)[2:ncol(data)],each = 2), rep(c("training", "test"), ncol(data)-1))
temp_all <- as.data.frame(temp_all)
temp_all

,인공신경망,의사결정나무,randomForest,SVM,다항로지스틱회귀분석
배급사점수 training,0.4545455,0.4025974,0.4545455,0.4545455,0.4025974
배급사점수 test,0.4025974,0.4025974,0.4025974,0.4025974,0.4025974
장르점수 training,0.4817001,0.4805195,0.4805195,0.4805195,0.4805195
장르점수 test,0.3589138,0.3636364,0.3506494,0.3636364,0.3636364
감독_배우시너지점수 training,0.5218418,0.4025974,0.5844156,0.4675325,0.4155844
감독_배우시너지점수 test,0.3872491,0.4025974,0.3612751,0.3766234,0.3636364
주연점수 training,0.6682409,0.6103896,0.8299882,0.6103896,0.5974026
주연점수 test,0.3530106,0.3636364,0.3435655,0.3506494,0.3636364
감독점수 training,0.6481700,0.5974026,0.7260921,0.5194805,0.4415584
감독점수 test,0.3093270,0.3116883,0.3105077,0.3116883,0.3246753


In [7]:
temp_all[seq(2,nrow(temp_all),2),]

,인공신경망,의사결정나무,randomForest,SVM,다항로지스틱회귀분석
배급사점수 test,0.4025974,0.4025974,0.4025974,0.4025974,0.4025974
장르점수 test,0.3589138,0.3636364,0.3506494,0.3636364,0.3636364
감독_배우시너지점수 test,0.3872491,0.4025974,0.3612751,0.3766234,0.3636364
주연점수 test,0.3530106,0.3636364,0.3435655,0.3506494,0.3636364
감독점수 test,0.3093270,0.3116883,0.3105077,0.3116883,0.3246753
개봉일스크린수 test,0.4025974,0.4025974,0.3246753,0.4025974,0.3766234
조연점수 test,0.3636364,0.3636364,0.3766234,0.3636364,0.3636364


In [8]:
subset(temp_all, temp_all$SVM == max(temp_all[seq(2,nrow(temp_all),2),"SVM"]))

,인공신경망,의사결정나무,randomForest,SVM,다항로지스틱회귀분석
배급사점수 test,0.4025974,0.4025974,0.4025974,0.4025974,0.4025974
개봉일스크린수 test,0.4025974,0.4025974,0.3246753,0.4025974,0.3766234


In [9]:
head(data)

cluster,배급사점수,장르점수,감독_배우시너지점수,주연점수,감독점수,개봉일스크린수,조연점수
1,2,10,3,4.000000,2.000000,339,2.600000
1,2,10,8,8.250000,4.000000,1354,3.250000
1,1,10,0,4.000000,3.000000,671,3.000000
1,3,10,3,6.777778,2.714286,575,3.000000
1,3,10,0,7.666667,2.500000,344,3.500000
1,2,20,6,6.500000,4.000000,549,3.166667


, 감독점수, 주연점수, 감독_배우시너지점수, 조연점수

In [281]:
result <- rep(0,5)
temp <- rep(0,5)
n <- 11
samp <- createDataPartition(data$cluster, p = 0.7, list = F)
for (i in 1:n) {


    # 학습용 데이터 셋
    data.tr <- subset(data[samp,], select = c(cluster, 배급사점수, 개봉일스크린수,  장르점수, 감독점수, 주연점수, 감독_배우시너지점수))

    # 검증용 데이터셋
    data.test <- subset(data[-samp,], select = c(cluster, 배급사점수,개봉일스크린수, 장르점수, 감독점수, 주연점수, 감독_배우시너지점수))

    # 문제 및 정답 만들기
    x <- subset(data.test, select = -cluster)
    y <- data.test$cluster
    result[1] <- result[1] + f.nnet(data.tr, x, y)
    result[2] <- result[2] + f.ctree(data.tr, x, y)
    result[3] <- result[3] + f.randomForest(data.tr, x, y)
    result[4] <- result[4] + f.svm(data.tr, x, y)
    result[5] <- result[5] + f.multinom(data.tr, x, y)
    
    temp[1] <- temp[1] + f.nnet(data.tr, subset(data.tr, select = -cluster), data.tr$cluster)
    temp[2] <- temp[2] + f.ctree(data.tr, subset(data.tr, select = -cluster), data.tr$cluster)
    temp[3] <- temp[3] + f.randomForest(data.tr, subset(data.tr, select = -cluster), data.tr$cluster)
    temp[4] <- temp[4] + f.svm(data.tr, subset(data.tr, select = -cluster), data.tr$cluster)
    temp[5] <- temp[5] + f.multinom(data.tr, subset(data.tr, select = -cluster), data.tr$cluster)
}
result <- result / n
temp <- temp / n

# weights:  103
initial  value 110.955815 
iter  10 value 83.356181
iter  20 value 80.582633
iter  30 value 79.976849
iter  40 value 79.842369
iter  50 value 79.840320
iter  60 value 78.134383
iter  70 value 77.989091
iter  80 value 77.986457
final  value 77.986454 
converged
# weights:  24 (14 variable)
initial  value 84.593146 
iter  10 value 69.249599
iter  20 value 52.279842
final  value 52.277085 
converged
# weights:  103
initial  value 85.818020 
final  value 83.394260 
converged
# weights:  24 (14 variable)
initial  value 84.593146 
iter  10 value 69.249599
iter  20 value 52.279842
final  value 52.277085 
converged
# weights:  103
initial  value 99.093060 
final  value 83.394259 
converged
# weights:  24 (14 variable)
initial  value 84.593146 
iter  10 value 69.249599
iter  20 value 52.279842
final  value 52.277085 
converged
# weights:  103
initial  value 88.506776 
final  value 83.394259 
converged
# weights:  24 (14 variable)
initial  value 84.593146 
iter  10 value 69.24959

In [282]:
res_all <- rbind(t(as.data.frame(temp)),
t(as.data.frame(result)))
colnames(res_all) <- c("인공신경망", "의사결정나무", "randomForest", "SVM", "다항로지스틱회귀분석")
rownames(res_all) <- c("트레이닝", "테스트")
res_all

,인공신경망,의사결정나무,randomForest,SVM,다항로지스틱회귀분석
트레이닝,0.4923259,0.6623377,1.0000000,0.7792208,0.6883117
테스트,0.3900293,0.5806452,0.6803519,0.7419355,0.7419355


In [283]:
head(data.tr)

,cluster,배급사점수,개봉일스크린수,장르점수,감독점수,주연점수,감독_배우시너지점수
1,1,2,339,10,2.000000,4.000000,3
2,1,2,1354,10,4.000000,8.250000,8
5,1,3,344,10,2.500000,7.666667,0
6,1,2,549,20,4.000000,6.500000,6
7,1,1,264,10,2.714286,6.777778,1
8,1,3,1180,10,2.500000,11.750000,5


In [120]:
str(data)

'data.frame':	108 obs. of  8 variables:
 $ cluster            : Factor w/ 3 levels "1","2","3": 1 1 1 1 1 1 1 1 1 1 ...
 $ 배급사점수         : int  2 2 1 3 3 2 1 3 1 3 ...
 $ 장르점수           : int  10 10 10 10 10 20 10 10 10 10 ...
 $ 감독_배우시너지점수: int  3 8 0 3 0 6 1 5 7 4 ...
 $ 주연점수           : num  4 8.25 4 6.78 7.67 ...
 $ 감독점수           : num  2 4 3 2.71 2.5 ...
 $ 개봉일스크린수     : int  339 1354 671 575 344 549 264 1180 513 825 ...
 $ 조연점수           : num  2.6 3.25 3 3 3.5 ...


In [14]:
cor(as.numeric(data$cluster), data[,2:ncol(data)])

배급사점수,장르점수,감독_배우시너지점수,주연점수,감독점수,개봉일스크린수,조연점수
0.1195046,0.293891,0.222731,0.6464251,0.4231618,0.2367386,0.7575004


In [15]:
cor(data[,2:ncol(data)])

,배급사점수,장르점수,감독_배우시너지점수,주연점수,감독점수,개봉일스크린수,조연점수
배급사점수,1.000000000,0.1424967,-0.05560081,0.2279586,0.07741333,-0.008884294,0.06128511
장르점수,0.142496688,1.0000000,0.16640569,0.1380842,0.20950164,0.288948774,0.36749476
감독_배우시너지점수,-0.055600808,0.1664057,1.00000000,0.1543465,0.31772470,0.355112344,0.33701061
주연점수,0.227958623,0.1380842,0.15434646,1.0000000,0.23063766,0.269348352,0.51109627
감독점수,0.077413333,0.2095016,0.31772470,0.2306377,1.00000000,0.087373655,0.36348825
개봉일스크린수,-0.008884294,0.2889488,0.35511234,0.2693484,0.08737366,1.000000000,0.23978602
조연점수,0.061285114,0.3674948,0.33701061,0.5110963,0.36348825,0.239786020,1.00000000
